In [ ]:
process = {
    "id": "absolute",
    "summary": "Absolute value",
    "description": "Computes the absolute value of a real number `x`, which is the \"unsigned\" portion of x and often denoted as *|x|*.\n\nThe no-data value `null` is passed through and therefore gets propagated.",
    "categories": [
      "math"
    ],
    "parameters": [
      {
        "name": "x",
        "description": "A number.",
        "schema": {
          "type": [
            "number",
            "null"
          ]
        }
      }
    ],
    "returns": {
      "description": "The computed absolute value.",
      "schema": {
        "type": [
          "number",
          "null"
        ],
        "minimum": 0
      }
    },
    "examples": [
      {
        "arguments": {
          "x": 0
        },
        "returns": 0
      },
      {
        "arguments": {
          "x": 3.5
        },
        "returns": 3.5
      },
      {
        "arguments": {
          "x": -0.4
        },
        "returns": 0.4
      },
      {
        "arguments": {
          "x": -3.5
        },
        "returns": 3.5
      }
    ],
    "links": [
      {
        "rel": "about",
        "href": "http://mathworld.wolfram.com/AbsoluteValue.html",
        "title": "Absolute value explained by Wolfram MathWorld"
      }
    ],
    "process_graph": {
      "lt": {
        "process_id": "lt",
        "arguments": {
          "x": {
            "from_parameter": "x"
          },
          "y": 0
        }
      },
      "multiply": {
        "process_id": "multiply",
        "arguments": {
          "x": {
            "from_parameter": "x"
          },
          "y": -1
        }
      },
      "if": {
        "process_id": "if",
        "arguments": {
          "value": {
            "from_node": "lt"
          },
          "accept": {
            "from_node": "multiply"
          },
          "reject": {
            "from_parameter": "x"
          }
        },
        "result": True
      }
    }
}
version = "1.0.0"

In [ ]:
import json
import uuid
from string import Template

class Foo:
    def _get_vue(self, component, parameters):
        libs = [
            "https://cdn.jsdelivr.net/npm/vue",
            "https://cdn.jsdelivr.net/npm/leaflet@1.6/dist/leaflet.js",
            "https://cdn.jsdelivr.net/npm/leaflet.antimeridian@1/dist/leaflet.antimeridian-src.min.js",
            "https://cdn.jsdelivr.net/npm/@openeo/vue-components@1.0.0-rc.4/assets/openeo-vue.umd.min.js"
        ]
        styles = [
            "https://cdn.jsdelivr.net/npm/leaflet@1.6/dist/leaflet.css",
            "https://cdn.jsdelivr.net/npm/@openeo/vue-components@1.0.0-rc.4/assets/openeo-vue.css"
        ]
        
        html = ""
        for lib in libs:
            html += '<script src="{}"></script>'.format(lib)
        for style in styles:
            html += '<link rel="stylesheet" href="{}">'.format(style)

        template = Template("""
        <div id="$uid"></div>
        <script>
          new Vue({
            el: '#$uid',
            render: h => h(window['openeo-vue'].$component, { 
              props: $parameters
            })
           });
        </script>
        """)
        html += template.substitute(
            uid = 'vue-' + uuid.uuid4().hex, # add prefix as HTML ids starting with numbers are problematic
            component = component,
            parameters = parameters
        )
        
        return html
    
    def _repr_html_(self):
        return self._get_vue(
            component = 'Process',
            parameters = json.dumps({
                "processData": process,
                "version": version
            })
        )
    
Foo()

In [ ]:
import openeo

con = openeo.connect("https://earthengine.openeo.org")

meta = con.collection_metadata("COPERNICUS/S2")
meta